<a href="https://colab.research.google.com/github/861728/ReserveService/blob/main/%ED%9A%8C%EC%82%AC_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%9D%B4%EB%A6%84_%EA%B2%80%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필수 라이브러리 및 최신 크롬 설치
!apt-get update
!apt-get remove -y chromium-browser chromium-chromedriver
!apt-get autoremove -y
!pip install selenium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,301 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3

In [ ]:
# 1. 필수 라이브러리 설치
!pip install selenium gspread google-auth chromedriver-autoinstaller

import time
import gspread
import chromedriver_autoinstaller
from google.colab import auth
from google.auth import default
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 2. 드라이버 및 인증
chromedriver_autoinstaller.install()
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

sheet_filename = '검수'
try:
    doc = gc.open(sheet_filename)
    worksheet = doc.sheet1
    print(f"✅ '{sheet_filename}' 연결 성공!")
except Exception as e:
    print(f"❌ 연결 실패: {e}")

# 3. 브라우저 최적화 설정 (이미지 로딩 차단 등)
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# --- 속도 향상을 위한 핵심 설정 ---
options.add_argument('--blink-settings=imagesEnabled=false') # 이미지 로딩 안 함
options.add_argument('--disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(3) # 요소 찾을 때 최대 3초만 대기

# 4. 검수 시작
rows = worksheet.get_all_values()
headers = rows[0]
data_rows = rows[1:]

print(f"⚡ 총 {len(data_rows)}개 매장 검수를 시작합니다 (속도 모드)")

# 결과 저장을 위한 리스트 (나중에 한 번에 업데이트)
updates = []

for i, row in enumerate(data_rows, start=2):
    if len(row) < 4: continue
    original_name = row[1].strip()
    url = row[3].strip()

    if not url or "naver.com" not in url: continue

    status_e = "" # 이름 변경 결과
    status_f = "" # URL 확인 결과

    try:
        driver.get(url)

        # iframe 전환 (최대 5초만 대기)
        try:
            WebDriverWait(driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

            # 이름 추출 (나타나면 즉시 복사)
            found_name = ""
            selectors = [".Fc73z", ".GHA9Y", ".yY7_V", ".v_v8D", "//div[@id='_title']//span"]

            for sel in selectors:
                try:
                    if sel.startswith("//"):
                        element = driver.find_element(By.XPATH, sel)
                    else:
                        element = driver.find_element(By.CSS_SELECTOR, sel)

                    if element.text.strip():
                        found_name = element.text.strip()
                        break
                except:
                    continue

            if found_name and original_name not in found_name:
                status_e = "이름 변경"

        except:
            if "/entry/place/" not in driver.current_url:
                status_f = "URL 확인 필요"

        # 결과 로그
        print(f"[{i}행] {original_name} 완료")

        # 일괄 업데이트를 위한 데이터 저장
        if status_e: updates.append({'range': f'E{i}', 'values': [[status_e]]})
        if status_f: updates.append({'range': f'F{i}', 'values': [[status_f]]})

        driver.switch_to.default_content()

    except Exception as e:
        print(f"[{i}행] 에러: {e}")

# 5. 시트 일괄 업데이트 (Batch Update)
if updates:
    worksheet.batch_update(updates)
    print(f"✅ 총 {len(updates)}건의 변경사항을 시트에 한 번에 기록했습니다.")

driver.quit()
print("🎉 검수 종료!")

✅ '검수' 연결 성공!
⚡ 총 7개 매장 검수를 시작합니다 (속도 모드)
[2행] 레스케이프 라망 시크레 완료
[3행] 소설한남 완료
[4행] 제로컴플렉스 완료
[5행] 익스퀴진 완료
[6행] 소울 완료
[7행] 메종조 퀴진 완료
[8행] 메종드라망 완료
✅ 총 1건의 변경사항을 시트에 한 번에 기록했습니다.
🎉 검수 종료!
